## Raw to csv preprocessing

In [1]:
from utility.preprocessing2 import clean_data
DATA_SIZE=100
BASE_PATH=f'data/{DATA_SIZE}'
clean_data(BASE_PATH)

process start..


  0%|          | 0/100 [00:00<?, ?it/s]

                                PatientID  AdmissionID  \
366  7A025E77-7832-4F53-B9A7-09A3F98AC17E            1   
367  7A025E77-7832-4F53-B9A7-09A3F98AC17E            2   
368  7A025E77-7832-4F53-B9A7-09A3F98AC17E            3   
369  7A025E77-7832-4F53-B9A7-09A3F98AC17E            4   
370  7A025E77-7832-4F53-B9A7-09A3F98AC17E            5   
371  7A025E77-7832-4F53-B9A7-09A3F98AC17E            6   
0    7A025E77-7832-4F53-B9A7-09A3F98AC17E            7   

          AdmissionStartDate         AdmissionEndDate  
366  1978-06-29 07:29:11.593  1978-07-10 04:05:51.787  
367  1982-10-14 07:01:43.613  1982-10-21 16:44:00.310  
368  1990-02-23 04:28:34.457  1990-03-13 15:55:29.837  
369  1994-05-15 11:34:00.840  1994-05-19 23:04:04.877  
370  2001-01-10 11:04:58.943  2001-01-27 00:54:18.730  
371  2001-12-29 20:28:01.050  2002-01-18 18:16:04.423  
0    2011-10-12 14:55:02.027  2011-10-22 01:16:07.557  
['1978-06-29 07:29:11.593', '1982-10-14 07:01:43.613', '1990-02-23 04:28:34.457', '1994

SystemExit: 

C:\Users\pedne\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Feature engineering 

In [2]:
from utility.preprocessing2 import clean_data,lower_case, preprocessing, upsample
from sklearn.model_selection import train_test_split
import pandas as pd
import os
DATA_SIZE=100
df=pd.read_csv(f"data/process/final_{DATA_SIZE}.csv")

In [3]:
# check data len 
DAYS=30
print(f"total Admission : {len(df)}")
print(f"admit within {DAYS} : {len(df[df['days']<=DAYS])} ")


total Admission : 272
admit within 30 : 3 


In [4]:
all_feature=['PatientGender', 'PatientRace','Age',
'PatientMaritalStatus', 'PatientLanguage',
'PatientPopulationPercentageBelowPoverty', 'AdmissionID',
'PrimaryDiagnosisCode', 'PrimaryDiagnosisDescription', 'Target', ]

### process_df and train_df we can use for evaluation

In [5]:
#convert df in lower case
df=lower_case(df)
process_df=preprocessing(df,DAYS,all_feature) # whole dataset you will for evaluation
train_df=upsample(process_df,300) #upsample low class

0    269
1      3
Name: Target, dtype: int64


In [6]:
train_df['Target'].value_counts()

1    300
0    269
Name: Target, dtype: int64

### Train model and save

In [9]:
from models.model2 import train_model
model,test,train = train_model(train_df)

Fitting 30 folds for each of 100 candidates, totalling 3000 fits
Precision: 1.0,Recall: 1.0
Model save..


### Evaluation 

In [10]:
from testing.evaluation2 import evaluation_data
# use train_df or process_df  set True to get csv for manual comparison 
df_test=evaluation_data(train_df.copy(),'save_model/model.pkl',True)

Precision: 0.9966777408637874,Recall: 1.0


In [11]:
# df_test=evaluation_data(train_df,'save_model/model.pkl')
print(f"total={len(df_test)}\n")
print(df_test.Target.value_counts())
print(f"\n0 <-> 0 = {len(df_test[(df_test['Target']==0) & (df_test['y_pred']==0)])}")
print(f"1 <-> 1 = {len(df_test[(df_test['Target']==1) & (df_test['y_pred']==1)])}")
print(f"0 <-> 1 = {len(df_test[(df_test['Target']==0) & (df_test['y_pred']==1)])}")
print(f"1 <-> 0 = {len(df_test[(df_test['Target']==1) & (df_test['y_pred']==0)])}")

total=569

1    300
0    269
Name: Target, dtype: int64

0 <-> 0 = 268
1 <-> 1 = 300
0 <-> 1 = 1
1 <-> 0 = 0


### inferences 

In [1]:
import pandas as pd
data={'PatientGender':['male','female'],
      'PatientRace':['white','white'],
      'Age':[41,32],
      'PatientMaritalStatus':['single','single'],
      'PatientLanguage':['english','english'],
      'PatientPopulationPercentageBelowPoverty':[11.53,12.3],
      'AdmissionID':[2,1],
      'PrimaryDiagnosisCode':['t70.3','t70.3'],
      'PrimaryDiagnosisDescription':['Caisson disease [decompression sickness]','Caisson']
     }
test_df=pd.DataFrame(data)

In [2]:
from testing.evaluation2 import infer
infer(pd.DataFrame(test_df))

[0 0]


['No Need To Admit', 'No Need To Admit']